로토 생성기 개발. 
간단 생성기 부터 랜덤 번호를 만들어 내고 높이 나온 빈도수 기반의 생성기 부터

머신 러닝 기반의 기존 당첨번호 학습 기반의 생성기 까지 개발 해 본다. 

1. 단순 생성기

In [1]:
import random

def addNumber(ns):
    n = random.randint(1,45)
    if n in ns : addNumber(ns)
    else : ns.append(n)

ns = []
for i in range(7): addNumber(ns)

print("2등 번호 : {}".format(ns.pop()))
print("1등 번호 : {}".format(ns))

2등 번호 : 43
1등 번호 : [40, 6, 30, 38, 3, 27]


랜덤 번호 기반의 제일 많이 나온 빈도수 기반

In [2]:
import random
import operator

NUNBER_LOOP = (956);

def addNumber(ns):
    n = random.randint(1,45)
    if n in ns : addNumber(ns)
    else : ns.append(n)

# 100회 실행하여 100회 번호를 획득
ns = []         
for i in range(NUNBER_LOOP) : 
    ns.append([])    
    for j in range(7): 
        addNumber(ns[i])

# 번호 출현 횟수를 기록
number_count = {}
for i in ns:
    for j in i : 
        if j in number_count : 
            tmp = number_count[j] + 1
            del number_count[j]
            number_count[j] = tmp
        else : 
            number_count[j] = 1

# 결과를 정렬. 값으로 키를 역정렬
number_count_sort = sorted(number_count.items(),key=operator.itemgetter(1),reverse=True)
print(number_count_sort)

# 7개의 리스트로 정리
final_number = []
for i in number_count_sort[:7]:
    final_number.append(i[0])

print("-------------- 결과 --------------------")
print("2등 번호 : {}".format(final_number.pop()))
print("1등 번호 : {}".format(sorted(final_number)))

[(11, 166), (42, 166), (28, 164), (25, 163), (34, 162), (12, 160), (6, 159), (17, 157), (10, 156), (24, 155), (30, 155), (23, 154), (13, 154), (27, 154), (33, 154), (3, 153), (16, 153), (1, 153), (5, 151), (44, 150), (45, 150), (39, 150), (8, 150), (40, 149), (14, 149), (19, 148), (7, 147), (29, 147), (32, 146), (15, 145), (9, 145), (37, 144), (18, 144), (20, 143), (35, 142), (4, 141), (41, 141), (21, 140), (43, 136), (38, 136), (22, 135), (31, 134), (26, 132), (36, 130), (2, 129)]
-------------- 결과 --------------------
2등 번호 : 6
1등 번호 : [11, 12, 25, 28, 34, 42]


역대 나온 로토 번호를 수집하여, 수집된 데이터 기반의 빈도수를 계산하여 번호 뽑기

In [11]:
import csv

# CSV 파일 경로
csv_path = "./data/lotto.csv"

ns = []

# CSV 파일 열기
with open(csv_path, newline="") as csv_file:
    # CSV 파일 읽기
    csv_reader = csv.reader(csv_file, delimiter=",")

    # 각 행(row)을 출력
    for row in csv_reader:
        ns+=[row]


print(ns)

# 번호 출현 횟수를 기록
number_count = {}
for i in ns:
    for j in i : 
        if j in number_count : 
            tmp = number_count[j] + 1
            del number_count[j]
            number_count[j] = tmp
        else : 
            number_count[j] = 1

# 결과를 정렬. 값으로 키를 역정렬
number_count_sort = sorted(number_count.items(),key=operator.itemgetter(1),reverse=True)
print(number_count_sort)

# 7개의 리스트로 정리
final_number = []
for i in number_count_sort[0:7]:
    final_number.append(i[0])

print("-------------- 결과 --------------------")
print("2등 번호 : {}".format(final_number.pop()))
print("1등 번호 : {}".format(sorted(final_number)))        

[['1059', '7', '10', '22', '25', '34', '40', '27'], ['1058', '11', '23', '25', '30', '32', '40', '42'], ['1057', '8', '13', '19', '27', '40', '45', '12'], ['1056', '13', '20', '24', '32', '36', '45', '29'], ['1055', '4', '7', '12', '14', '22', '33', '31'], ['1054', '14', '19', '27', '28', '30', '45', '33'], ['1053', '22', '26', '29', '30', '34', '45', '15'], ['1052', '5', '17', '26', '27', '35', '38', '1'], ['1051', '21', '26', '30', '32', '33', '35', '44'], ['1050', '6', '12', '31', '35', '38', '43', '17'], ['1049', '3', '5', '13', '20', '21', '37', '17'], ['1048', '6', '12', '17', '21', '32', '39', '30'], ['1047', '2', '20', '33', '40', '42', '44', '32'], ['1046', '7', '16', '25', '29', '35', '36', '28'], ['1045', '6', '14', '15', '19', '21', '41', '37'], ['1044', '12', '17', '20', '26', '28', '36', '4'], ['1043', '3', '5', '12', '22', '26', '31', '19'], ['1042', '5', '14', '15', '23', '34', '43', '4'], ['1041', '6', '7', '9', '11', '17', '18', '45'], ['1040', '8', '16', '26', '29', 

* ML 을 이용하여, 학습시키고 다음주 출연 빈도가 높은 숫자를 예측해 본다. 
* 어떤 모델링을 써야 할지 부터 고민해 본다.

In [22]:
# 필요한 라이브러리 import
import tensorflow as tf
import pandas as pd
import numpy as np

# Step 1: 데이터 수집
# 이전 추첨 번호 데이터 다운로드
url = './data/lotto.csv'
lotto_numbers = pd.read_csv(url)


      No1  No2  No3  No4  No5  No6
0       7   10   22   25   34   40
1      11   23   25   30   32   40
2       8   13   19   27   40   45
3      13   20   24   32   36   45
4       4    7   12   14   22   33
...   ...  ...  ...  ...  ...  ...
1054   16   24   29   40   41   42
1055   14   27   30   31   40   42
1056   11   16   19   21   27   31
1057    9   13   21   25   32   42
1058   10   23   29   33   37   40

[1059 rows x 6 columns]


In [25]:

# Step 2: 데이터 전처리
# 로또 번호를 0과 1로 변환하는 함수 정의
def transform_numbers(numbers):
    result = np.zeros(45)
    result[numbers-1] = 1
    return result

# 훈련 데이터와 테스트 데이터 처리
X = lotto_numbers.iloc[:1,1:].apply(transform_numbers, axis=1).values
y = lotto_numbers.iloc[1:,1:].apply(transform_numbers, axis=1).values

print(X)
print(y)


[array([0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])                       ]


In [26]:

# Step 3: 모델 학습
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(45,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=100)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:

# Step 4: 모델 예측
# 마지막 추첨 번호 입력
last_numbers = lotto_numbers.iloc[1:].apply(transform_numbers, axis=1).values
# 모델 예측
predictions = model.predict(last_numbers)
# 확률이 가장 높은 6개의 번호 출력
result = np.argsort(predictions[0])[::-1][:6]+1
print(result)